In [4]:
import keras
keras.__version__

'2.2.4'

# Data Preparation

In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, 10)# convert class vectors to binary class matrices

<a href="https://keras.io/datasets/">datasets</a> (all uint8):
* mnist: X.shape is (N, 28, 28); y.shape is (N,) from 0 to 9
* cifar10: X.shape is (N, 3, 32, 32); y.shape is (N,) from 0 to 9
* cifar100: X.shape is (N, 3, 32, 32); y.shape is (N,) of string labels
 * label_mode: "fine" or "coarse".
* imdb: 
* reuters

In [7]:
from keras import backend as K
if K.image_dim_ordering() == 'th':
    X_train_ = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test_ = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
elif  K.image_dim_ordering() == 'tf':
    X_train_ = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test_ = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

NameError: name 'img_rows' is not defined

Conform with Convolution2D's requirements for different backends

In [ ]:
try: from sklearn.model_selection import train_test_split
except: from sklearn.cross_validation import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.15, random_state=0)

in general not necessary as model.fit can specify validation ratio

# Model Building

## Sequential Building

In [1]:
from keras.models import Sequential
model = Sequential()

Using TensorFlow backend.


In [ ]:
from keras.layers import Dense
model.add(Dense(512, input_shape=(96,96)))

<a href="https://keras.io/layers/core/#dense">Dense</a>(output_dim, input_shape, init, W_regularizer)
* output_dim: Linear channel's length
* input_shape: only necessary on first layer; otherwise is inferred internally
* init: "glorot_normal"
* W_regularizer=l2(0.1) [from keras.regularizers import l2]
* activation: if specified, equiv to addling the Activation layer as the next item

<a href="https://keras.io/layers/core/#activation">Activation</a>(<a href="https://keras.io/activations/">activation</a>)
* 'relu', 'softplus', 'softsign', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'softmax'(3dim or 2dim)
* custom function with 1 input & 1 output

<a href="https://keras.io/layers/convolutional/#convolution2d">Convolution2D</a>(nb_filter, nb_row, nb_col, init='glorot_uniform', activation='linear', weights=None, border_mode='valid', subsample=(1, 1), dim_ordering='default', W_regularizer=None, b_regularizer=None, activity_regularizer=None, W_constraint=None, b_constraint=None, bias=True)
* Input shape: 4D tensor with shape: (samples, channels, rows, cols) if dim_ordering='th' or 4D tensor with shape: (samples, rows, cols, channels) if dim_ordering='tf'.

MaxPooling2D(pool_size=pool_size)

merge([tower_1, tower_2, tower_3], mode='concat', concat_axis=1)

Flatten()

<a href="https://keras.io/layers/wrappers/#timedistributed">TimeDistributed</a>(Dense(10,input_shape))
* input must be at least 3D (batchSize x timeSteps x DenseInput)

## Functional API ~nngraph

https://keras.io/getting-started/functional-api-guide/

In [ ]:
from keras.layers import Input, TimeDistributed
from keras.layers import Dense, ...

In [ ]:
inputs = Input(shape=(100,), dtype='int32')#Sequential
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(input=inputs, output=predictions)

In [ ]:
x = Input(shape=(dim1, dim2, dim3))
a = TimeDistributed(LSTM(output_dim=10))(x) #apply "LSTM" model to each of dim1's elements
b = LSTM(col_hidden)(a)
prediction = Dense(nb_classes, activation='softmax')(b)
model = Model(input=x, output=prediction)

In [ ]:
model = Model(input=[main_input, auxiliary_input], output=[main_output, auxiliary_output]) #multi-input/outputs
model.compile(optimizer='rmsprop', loss='binary_crossentropy', loss_weights=[1., 0.2])
model.fit([headline_data, additional_data], [labels, labels],nb_epoch=50, batch_size=32)
# if names are provided for the inputs/outputs:
model.compile(optimizer='rmsprop',loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
                          loss_weights={'main_output': 1., 'aux_output': 0.2})
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          nb_epoch=50, batch_size=32)

## Common Model Properties

In [ ]:
model.summary()  #human-readable
model.output_shape # shape of output

In [ ]:
from keras.models import model_from_json
model.save_weights("model.h5", overwrite=True)
with open("model.json", "w") as outfile:  #uncompiled
    json.dump(model.to_json(), outfile)
with open("model.json", "r") as jfile:
    model = model_from_json(json.load(jfile))
model.load_weights("model.h5")
model.compile("sgd", "mse")

In [ ]:
model.add(Dense(64, activation='relu', name="dense_one"))
model.get_layer("dense_one")  #get layer by name

## Common Layer Properties

In [ ]:
layer = model.layers[0]  #extract the first layer from the list of model's layers
layer.get_weights()[0] # For "Dense": 0 for weights, 1 for bias
layer.set_weights(weights) # sets the weights of the layer from a list of Numpy arrays 
layer.get_config()# returns a dictionary containing the configuration of the layer. 

If a layer has a single node (i.e. if it isn't a shared layer), you can get its input tensor, output tensor, input shape and output shape via:

    layer.input
    layer.output
    layer.input_shape
    layer.output_shape

If the layer has multiple nodes (see: the concept of layer node and shared layers), you can use the following methods:

    layer.get_input_at(node_index)
    layer.get_output_at(node_index)
    layer.get_input_shape_at(node_index)
    layer.get_output_shape_at(node_index)


In [ ]:
from keras.optimizers import SGD, Adam, RMSprop, ...
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])  

<a href="https://keras.io/models/model/">Parameters</a>:
* loss: 'categorical_crossentropy' [target y is of shape (N,nclass), i.e. one-hot encoding], 'mse' (mean_squared_error), 'mae', 'mape', 'msle', 'sparse_categorical_crossentropy' (accept sparse labels), squared_hinge, hinge, binary_crossentropy, kld (kullback_leibler_divergence), 'poisson' [Mean of (predictions - targets * log(predictions))], 'cosine_proximity'
* <a href="https://keras.io/optimizers/">optimizer</a>: common parameters are clipnorm (max allowed gradient 2-norm) and clipvalue (max allowed gradient 1-norm)
 * 'sgd' or SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
 * 'adagrad' or Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
 * 'rmsprop' or RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
 * 'adadelta' or Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
 * 'adam' or Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
 * 'adamax' or Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
 * 'nadam' or Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
* metrics: list of metrics to be evaluated by the model during training and testing
 * 'accuracy'
* sample_weight_mode

# Training

In [ ]:
history = model.fit(x,y,batch_size=32, nb_epoch=10,verbose=1,callbacks=[], validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None)

<a href="https://keras.io/models/model/">Parameters</a>:

* x: Numpy array of training data, or list of Numpy arrays if the model has multiple inputs. If all inputs in the model are named, you can also pass a dictionary mapping input names to Numpy arrays.
* y: Numpy array of target data, or list of Numpy arrays if the model has multiple outputs. If all outputs in the model are named, you can also pass a dictionary mapping output names to Numpy arrays.
* batch_size: integer. Number of samples per gradient update.
* nb_epoch: integer, the number of times to iterate over the training data arrays.
* verbose: 0, 1, or 2. Verbosity mode. 0 = silent, 1 = verbose, 2 = one log line per epoch.
* validation_split: float between 0 and 1: fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch.
* validation_data: data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data. This could be a tuple (x_val, y_val) or a tuple (val_x, val_y, val_sample_weights).
* shuffle: boolean, whether to shuffle the training data before each epoch.
* class_weight: optional dictionary mapping class indices (integers) to a weight (float) to apply to the model's loss for the samples from this class during training. This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
* sample_weight: optional array of the same length as x, containing weights to apply to the model's loss for each sample. In the case of temporal data, you can pass a 2D array with shape (samples, sequence_length), to apply a different weight to every timestep of every sample. In this case you should make sure to specify sample_weight_mode="temporal" in compile().
* callbacks: list of callbacks to be called during training. See callbacks.

In [ ]:
# General model() class
evaluate(self, x, y, batch_size=32, verbose=1, sample_weight=None)
predict(self, x, batch_size=32, verbose=0)
train_on_batch(self, x, y, sample_weight=None, class_weight=None)
test_on_batch(self, x, y, sample_weight=None)
predict_on_batch(self, x)
fit_generator(self, generator, samples_per_epoch, nb_epoch, verbose=1, callbacks=[], validation_data=None, nb_val_samples=None, class_weight={}, max_q_size=10, nb_worker=1, pickle_safe=False)
evaluate_generator(self, generator, val_samples, max_q_size=10, nb_worker=1, pickle_safe=False)
predict_generator(self, generator, val_samples, max_q_size=10, nb_worker=1, pickle_safe=False)
get_layer(self, name=None, index=None)

https://keras.io/models/model/

In [ ]:
#Sequential() only
predict_classes(self, x, batch_size=32, verbose=1)
predict_proba(self, x, batch_size=32, verbose=1)

https://keras.io/models/sequential/

# Custom Evaluation

In [ ]:
y_hat = model.predict_classes(x) #Sequential() only
pandas.crosstab(y_hat, y_test)

In [ ]:
#Visualize wrong data
test_wrong = [im for im in zip(X_test_,y_hat,y_test) if im[1] != im[2]]
for ind, val in enumerate(test_wrong[:100]):
    plt.subplot(10, 10, ind + 1)
    im = 1 - val[0].reshape((img_rows, img_cols,))
    axis("off")
    plt.imshow(im, cmap='gray')

In [ ]:
#visualize trained weights
W1 = model.layers[0].get_weights()[0] # 0 for weights, 1 for bias
print("weight matrix transposed:",W1.T.shape)
plt.figure(figsize=(5, 5))
for i in range(100):
    plt.subplot(10, 10, i + 1)
    im = W1.T[i].reshape((img_rows, img_cols,))
    plt.axis("off")
    plt.imshow(im, cmap='seismic')